In [2]:
import cv2
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image

Matplotlib is building the font cache; this may take a moment.


In [ ]:
#load images from folder
#augment images with transforms here :)
train_datagen = ImageDataGenerator(
    rescale=1/255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
)

train_generator = train_datagen.flow_from_directory("./training",target_size=(300,300),class_mode='binary')

#validation data, used while training
validation_datagen = ImageDataGenerator(rescale=1/255)
validation_generator = train_datagen.flow_from_directory(
    "./validation",
    target_size=(300,300),
    class_mode='binary',
)

#callback to stop training when 80% accuracy reached on validation set
class myCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self,epoch,logs={}):
        if(logs.get('val_accuracy')>0.80):
            print("\nAccuracy reached 80%, stopping training")
            self.model.stop_training = True
callbacks = myCallback()

In [ ]:
#create model
model = tf.keras.models.Sequential([
    #a bunch of convolution and pooling layers to filter down the image to essential features (hopefully)
    tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(300,300,3)),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    #dropout for regularization (this really impacted model performance)
    tf.keras.layers.Dropout(0.2),
    #sigmoid drives values towards 0 and 1 for binary classification problems
    tf.keras.layers.Dense(1,activation='sigmoid'),
])
model.summary()

In [ ]:
#compile model with rmsprop optimizer
model.compile(loss='binary_crossentropy',
              optimizer=tf.keras.optimizers.RMSprop(learning_rate=0.001),
              metrics=['accuracy'])

In [ ]:
#train model
history = model.fit(train_generator, epochs=20, validation_data=validation_generator, callbacks=callbacks)

In [ ]:
#test on custom images
img = image.load_img("./custom/visible/2.png", target_size=(300,300))
x = image.img_to_array(img)
x = np.expand_dims(x,axis=0)

image_tensor = np.vstack([x])
classes = model.predict(image_tensor)

print(train_generator.class_indices)
print(classes)
if classes[0]>0.5:
    print("is visible")
else:
    print("is not visible")

In [3]:
#Take video input and use the model to determine if an intruder exists
vid = cv2.VideoCapture("./out.mp4")
while(vid.isOpened()):
    ret, frame = vid.read()
    if ret == True:
        cv2.imshow('frame',frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    else:
        vid.release()
        cv2.destroyAllWindows()
        break
vid.release()
cv2.destroyAllWindows()

#intruder = False
#video -> frames -> model(frame)
#if model(frame) == visible:
#intruder = True
#else:
#continue
#return intruder

2024-07-29 16:03:26.777 Python[39484:1582767] WARNING: Secure coding is not enabled for restorable state! Enable secure coding by implementing NSApplicationDelegate.applicationSupportsSecureRestorableState: and returning YES.


In [ ]:
#save model to disk
model.save("SecurityModel.keras")

In [ ]:
#load it and create a single function that takes a file and outputs a prediction
loaded_model = tf.keras.models.load_model('SecurityModelFinal.keras')

def classify(imgPath):
    #preprocess image into tensor
    img = image.load_img(imgPath, target_size=(300,300))
    x = image.img_to_array(img)
    x = np.expand_dims(x,axis=0)
    tensor = np.vstack([x])
    #classify tensor
    classes = model.predict(tensor)
    if classes[0]>0.5:
        print("model prediction: is visible")
        return True
    else:
        print("model prediction: is not visible")
        return False


In [ ]:
#use loaded_model
classify("custom/visible/3.png")
classify("custom/not_visible/5.png")